# General instructions

The goal of the project is to materialize a set of **exploratory workloads** over a real-world, large-scale,  open-domain KG: [WikiData](https://www.wikidata.org/wiki/Wikidata:Main_Page)

An exploratory workload  is composed by a set of queries, where each query is related to the information obtained previously.

An exploratory workload starts with a usually vague, open ended question, and does not assume the person issuing the workload has a clear understanding of the data contained in the target database or its structure.

Remeber that:

1. All the queries must run in the python notebook
2. You can use classes and properties only if you find them via a SPARQL query that must be present in the notebook
3. You do not delete useless queries. Keep everything that is synthatically valid 

```
?p <http://schema.org/name> ?name .
```
    
    is the BGP returning a human-readable name of a property or a class in Wikidata.
    
    

In [3]:
## SETUP used later

from SPARQLWrapper import SPARQLWrapper, JSON


prefixString = """
##-f5249dead5-##
PREFIX wd: <http://www.wikidata.org/entity/> 
PREFIX wdt: <http://www.wikidata.org/prop/direct/> 
PREFIX sc: <http://schema.org/>
"""

# select and construct queries
def run_query(queryString,verbose = True):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-03.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        results = sparql.query()
        json_results = results.convert()
        if len(json_results['results']['bindings'])==0:
            print("Empty")
            return []
        array = []
        for bindings in json_results['results']['bindings']:
            app =  [ (var, value['value'])  for var, value in bindings.items() ] 
            if verbose:
                print( app)
            array.append(app)
        if verbose:
            print(len(array))
        return array

    except Exception as e :
        print("The operation failed", e)
    
# ASk queries
def run_ask_query(queryString):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-03.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        return sparql.query().convert()

    except Exception as e :
        print("The operation failed", e)

# Movie Workflow Series ("Tv series HIMYM explorative search") 


Consider the following exploratory scenario:


> we are interested in the TV series "How I met your mother" and we want to investigate the main aspects related to the actors and directors involved in the production, know the numerber of seasons and check what are the episodes which got the higher success/impact.


## Useful URIs for the current workflow
The following are given:

| IRI           | Description   | Role      |
| -----------   | -----------   |-----------|
| `wdt:P1647`   | subproperty   | predicate |
| `wdt:P31`     | instance of   | predicate |
| `wdt:P106`    | profession    | predicate | 
| `wdt:P279`    | subclass      | predicate |
| `wdt:P4969`    | derivative work      | predicate |
| `wd:Q147235` | How I met your mother        | node |
| `wd:Q23831` | The Office (US)        | node |



Also consider

```
wd:Q23831 ?p ?obj .
```

is the BGP to retrieve all **properties of The Office (US)**

Please consider that when you return a resource, you should return the IRI and the label of the resource. In particular, when the task require you to identify a BGP the result set must always be a list of couples IRI - label.

The workload should

1. Identify the BGP for tv series

2. Return the number of seasons and episodes per season of the tv series (the result set must be triples of season IRI, label and #episodes).

3. Get the number of episodes in which the cast members played a role. Who are the most present actors? (the result set must be a list of triples actor/actress IRI, label and #episodes)

4. Check who is the actor who acted in more films while working on "How I met your mother" (the result set must be a list of triples actor/actress IRI, label and #films).

5. Compare HIMYM with the tv series "The Office (US)" in terms of number of seasons, episods and cast members (the result set must be two elements -one for each tv series- of tv series IRI, label, #seasons, #episodes and #cast members).

6. Return how many of the actors who are members of the cast of the tv series have [Kavin Bacon number](https://en.wikipedia.org/wiki/Six_Degrees_of_Kevin_Bacon#:~:text=Kevin%20Bacon%20himself%20has%20a,Bacon%20number%20is%20N%2B1.) equal to 2 (the result set must be a list of couples actor/actress IRI and label).

7. Consider the actors who are members of the cast of HIMYM. Amongst the tv series which these actors acted return only those which received more than 2 awards (the result set must be triples of tv series IRI, label, #awards won).

## Task 1
Identify the BGP for tv series

In [4]:
# query example
#Q23831 the Office
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   wd:Q23831 ?p ?obj .
   # get the label
   ?p sc:name ?name.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
The operation failed timed out


Final query for this task

In [5]:
# query example
#Q23831 the Office
queryString = """
SELECT DISTINCT ?obj ?objName
WHERE {
   # bind something
   wd:Q23831 wdt:P31 ?obj .
   # get the label
   ?obj sc:name ?objName.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q5398426'), ('objName', 'television series')]
1


## Task 2
Return the number of seasons and episodes per season of the tv series (the result set must be triples of season IRI, label and #episodes).

In [18]:

# Q147235	How I met your mother
queryString = """
SELECT DISTINCT ?p ?name ?obj ?objName 
WHERE {
   # bind something
   wd:Q147235 ?p ?obj .
   
   # get the label
   ?p sc:name ?name
   optional{?obj sc:name ?objName.}
   filter regex(?objName, "season").
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P527'), ('name', 'has part'), ('obj', 'http://www.wikidata.org/entity/Q2438066'), ('objName', 'How I Met Your Mother, season 6')]
[('p', 'http://www.wikidata.org/prop/direct/P527'), ('name', 'has part'), ('obj', 'http://www.wikidata.org/entity/Q2715578'), ('objName', 'How I Met Your Mother, season 1')]
[('p', 'http://www.wikidata.org/prop/direct/P527'), ('name', 'has part'), ('obj', 'http://www.wikidata.org/entity/Q13567027'), ('objName', 'How I Met Your Mother, season 9')]
[('p', 'http://www.wikidata.org/prop/direct/P527'), ('name', 'has part'), ('obj', 'http://www.wikidata.org/entity/Q2567330'), ('objName', 'How I Met Your Mother, season 4')]
[('p', 'http://www.wikidata.org/prop/direct/P527'), ('name', 'has part'), ('obj', 'http://www.wikidata.org/entity/Q338715'), ('objName', 'How I Met Your Mother, season 8')]
[('p', 'http://www.wikidata.org/prop/direct/P527'), ('name', 'has part'), ('obj', 'http://www.wikidata.org/entity/Q582332'

In [23]:

# Q147235	How I met your mother
queryString = """
SELECT DISTINCT ?p ?name ?obj ?objName 
WHERE {
   # bind something
   wd:Q147235 ?p ?obj .
  
   
   # get the label
   ?p sc:name ?name
   optional{?obj sc:name ?objName.}
   filter regex(?objName, "season").
}
ORDER BY ASC (?objName)
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P527'), ('name', 'has part'), ('obj', 'http://www.wikidata.org/entity/Q2715578'), ('objName', 'How I Met Your Mother, season 1')]
[('p', 'http://www.wikidata.org/prop/direct/P527'), ('name', 'has part'), ('obj', 'http://www.wikidata.org/entity/Q3468515'), ('objName', 'How I Met Your Mother, season 2')]
[('p', 'http://www.wikidata.org/prop/direct/P527'), ('name', 'has part'), ('obj', 'http://www.wikidata.org/entity/Q2555117'), ('objName', 'How I Met Your Mother, season 3')]
[('p', 'http://www.wikidata.org/prop/direct/P527'), ('name', 'has part'), ('obj', 'http://www.wikidata.org/entity/Q2567330'), ('objName', 'How I Met Your Mother, season 4')]
[('p', 'http://www.wikidata.org/prop/direct/P527'), ('name', 'has part'), ('obj', 'http://www.wikidata.org/entity/Q582332'), ('objName', 'How I Met Your Mother, season 5')]
[('p', 'http://www.wikidata.org/prop/direct/P527'), ('name', 'has part'), ('obj', 'http://www.wikidata.org/entity/Q2438066'

In [22]:
# Q2567330 'How I Met Your Mother, season 4'
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   wd:Q2567330 ?p ?obj .
   # get the label
   ?p sc:name ?name.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)


Results
[('p', 'http://www.wikidata.org/prop/direct/P1113'), ('name', 'number of episodes')]
[('p', 'http://www.wikidata.org/prop/direct/P1258'), ('name', 'Rotten Tomatoes ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1712'), ('name', 'Metacritic ID')]
[('p', 'http://www.wikidata.org/prop/direct/P179'), ('name', 'part of the series')]
[('p', 'http://www.wikidata.org/prop/direct/P2529'), ('name', 'ČSFD film ID')]
[('p', 'http://www.wikidata.org/prop/direct/P2638'), ('name', 'TV.com ID')]
[('p', 'http://www.wikidata.org/prop/direct/P2704'), ('name', 'EIDR content ID')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('name', 'instance of')]
[('p', 'http://www.wikidata.org/prop/direct/P3302'), ('name', 'Open Media Database film ID')]
[('p', 'http://www.wikidata.org/prop/direct/P364'), ('name', 'original language of film or TV show')]
[('p', 'http://www.wikidata.org/prop/direct/P437'), ('name', 'distribution format')]
[('p', 'http://www.wikidata.org/prop/direct/P527'), ('name', 'ha

Final query for this task

In [25]:
# write your final query
# P1113 number of episodes
# Q147235	How I met your mother
queryString = """
SELECT DISTINCT ?p ?name ?obj ?objName ?numeberEpisode 
WHERE {
   # bind something
   wd:Q147235 ?p ?obj .
   ?obj wdt:P1113 ?numeberEpisode.
   
   # get the label
   ?p sc:name ?name
   optional{?obj sc:name ?objName.}
   filter regex(?objName, "season").
}
ORDER BY ASC (?objName)
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P527'), ('name', 'has part'), ('obj', 'http://www.wikidata.org/entity/Q2715578'), ('objName', 'How I Met Your Mother, season 1'), ('numeberEpisode', '22')]
[('p', 'http://www.wikidata.org/prop/direct/P527'), ('name', 'has part'), ('obj', 'http://www.wikidata.org/entity/Q3468515'), ('objName', 'How I Met Your Mother, season 2'), ('numeberEpisode', '22')]
[('p', 'http://www.wikidata.org/prop/direct/P527'), ('name', 'has part'), ('obj', 'http://www.wikidata.org/entity/Q2555117'), ('objName', 'How I Met Your Mother, season 3'), ('numeberEpisode', '20')]
[('p', 'http://www.wikidata.org/prop/direct/P527'), ('name', 'has part'), ('obj', 'http://www.wikidata.org/entity/Q2567330'), ('objName', 'How I Met Your Mother, season 4'), ('numeberEpisode', '24')]
[('p', 'http://www.wikidata.org/prop/direct/P527'), ('name', 'has part'), ('obj', 'http://www.wikidata.org/entity/Q582332'), ('objName', 'How I Met Your Mother, season 5'), ('numeberEpisode', 

## Task 3
Get the number of episodes in which the cast members played a role. Who are the most present actors? (the result set must be a list of triples actor/actress IRI, label and #episodes)

In [29]:
# write your queries
# Q147235	How I met your mother
queryString = """
SELECT DISTINCT ?p ?name ?obj ?objName 
WHERE {
   # bind something
   wd:Q147235 ?p ?obj .
  
   
   # get the label
   ?p sc:name ?name
   optional{?obj sc:name ?objName.}
   filter regex(?name, "cast").
}
ORDER BY ASC (?objName)
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P161'), ('name', 'cast member'), ('obj', 'http://www.wikidata.org/entity/Q8349'), ('objName', '"Weird Al" Yankovic')]
[('p', 'http://www.wikidata.org/prop/direct/P161'), ('name', 'cast member'), ('obj', 'http://www.wikidata.org/entity/Q24576840'), ('objName', 'Aaron Hendry')]
[('p', 'http://www.wikidata.org/prop/direct/P161'), ('name', 'cast member'), ('obj', 'http://www.wikidata.org/entity/Q667979'), ('objName', 'Aaron Hill')]
[('p', 'http://www.wikidata.org/prop/direct/P161'), ('name', 'cast member'), ('obj', 'http://www.wikidata.org/entity/Q2708237'), ('objName', 'Abby Elliott')]
[('p', 'http://www.wikidata.org/prop/direct/P161'), ('name', 'cast member'), ('obj', 'http://www.wikidata.org/entity/Q50736659'), ('objName', 'Abhi Sinha')]
[('p', 'http://www.wikidata.org/prop/direct/P161'), ('name', 'cast member'), ('obj', 'http://www.wikidata.org/entity/Q321131'), ('objName', 'Abigail Spencer')]
[('p', 'http://www.wikidata.org/prop/dire

In [36]:
#P161  'cast member'
#Q131332 'Amanda Peet'
queryString = """
SELECT DISTINCT ?p ?name ?obj ?objName 
WHERE {
   # bind something
   wd:Q131332 ?p ?obj .
  
   
   # get the label
   ?p sc:name ?name
   optional{?obj sc:name ?objName.}
   
}
ORDER BY ASC (?objName)
LIMIT 100
"""

print("Results")
x=run_query(queryString)


Results
[('p', 'http://www.wikidata.org/prop/direct/P735'), ('name', 'given name'), ('obj', 'http://www.wikidata.org/entity/Q453722'), ('objName', 'Amanda')]
[('p', 'http://www.wikidata.org/prop/direct/P140'), ('name', 'religion'), ('obj', 'http://www.wikidata.org/entity/Q5043'), ('objName', 'Christianity')]
[('p', 'http://www.wikidata.org/prop/direct/P69'), ('name', 'educated at'), ('obj', 'http://www.wikidata.org/entity/Q49088'), ('objName', 'Columbia University')]
[('p', 'http://www.wikidata.org/prop/direct/P26'), ('name', 'spouse'), ('obj', 'http://www.wikidata.org/entity/Q503997'), ('objName', 'David Benioff')]
[('p', 'http://www.wikidata.org/prop/direct/P1412'), ('name', 'languages spoken, written or signed'), ('obj', 'http://www.wikidata.org/entity/Q1860'), ('objName', 'English')]
[('p', 'http://www.wikidata.org/prop/direct/P69'), ('name', 'educated at'), ('obj', 'http://www.wikidata.org/entity/Q5504210'), ('objName', 'Friends Seminary')]
[('p', 'http://www.wikidata.org/prop/dir

In [34]:
#Q13567027 'How I Met Your Mother, season 9'

queryString = """
SELECT DISTINCT ?p ?name ?obj ?objName 
WHERE {
   # bind something
   wd:Q13567027 ?p ?obj .
  
   
   # get the label
   ?p sc:name ?name
   optional{?obj sc:name ?objName.}
   
   
}
ORDER BY ASC (?objName)
LIMIT 100
"""

print("Results")
x=run_query(queryString)


Results
[('p', 'http://www.wikidata.org/prop/direct/P2684'), ('name', 'Kijkwijzer rating'), ('obj', 'http://www.wikidata.org/entity/Q23649982'), ('objName', '9')]
[('p', 'http://www.wikidata.org/prop/direct/P527'), ('name', 'has part'), ('obj', 'http://www.wikidata.org/entity/Q16746256'), ('objName', 'Bass Player Wanted')]
[('p', 'http://www.wikidata.org/prop/direct/P527'), ('name', 'has part'), ('obj', 'http://www.wikidata.org/entity/Q16746317'), ('objName', 'Bedtime Stories')]
[('p', 'http://www.wikidata.org/prop/direct/P527'), ('name', 'has part'), ('obj', 'http://www.wikidata.org/entity/Q16746145'), ('objName', 'Coming Back')]
[('p', 'http://www.wikidata.org/prop/direct/P527'), ('name', 'has part'), ('obj', 'http://www.wikidata.org/entity/Q16738646'), ('objName', 'Daisy')]
[('p', 'http://www.wikidata.org/prop/direct/P364'), ('name', 'original language of film or TV show'), ('obj', 'http://www.wikidata.org/entity/Q1860'), ('objName', 'English')]
[('p', 'http://www.wikidata.org/prop/

In [42]:
# Q16746256  'Bass Player Wanted'
queryString = """
SELECT DISTINCT ?p ?name ?obj ?objName 
WHERE {
   
   wd:Q16746256 ?p ?obj .
  
   # get the label
   ?p sc:name ?name
   optional{?obj sc:name ?objName.}
   
   
}
ORDER BY ASC (?objName)
LIMIT 100
"""

print("Results")
x=run_query(queryString)


Results
[('p', 'http://www.wikidata.org/prop/direct/P364'), ('name', 'original language of film or TV show'), ('obj', 'http://www.wikidata.org/entity/Q1860'), ('objName', 'English')]
[('p', 'http://www.wikidata.org/prop/direct/P179'), ('name', 'part of the series'), ('obj', 'http://www.wikidata.org/entity/Q147235'), ('objName', 'How I Met Your Mother')]
[('p', 'http://www.wikidata.org/prop/direct/P4908'), ('name', 'season'), ('obj', 'http://www.wikidata.org/entity/Q13567027'), ('objName', 'How I Met Your Mother, season 9')]
[('p', 'http://www.wikidata.org/prop/direct/P57'), ('name', 'director'), ('obj', 'http://www.wikidata.org/entity/Q460221'), ('objName', 'Pamela Fryman')]
[('p', 'http://www.wikidata.org/prop/direct/P156'), ('name', 'followed by'), ('obj', 'http://www.wikidata.org/entity/Q15846040'), ('objName', 'Slapsgiving 3: Slappointment in Slapmarra')]
[('p', 'http://www.wikidata.org/prop/direct/P155'), ('name', 'follows'), ('obj', 'http://www.wikidata.org/entity/Q16746318'), ('

In [44]:
# Q21191270  'television series episode' 
#P161 'cast member'
queryString = """
SELECT DISTINCT ?p ?name ?obj ?objName 
WHERE {
   
   wd:Q21191270 ?p ?obj .
  
   # get the label
   ?p sc:name ?name
   optional{?obj sc:name ?objName.}
   
   
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)


Results
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('name', 'properties for this type'), ('obj', 'http://www.wikidata.org/entity/P31'), ('objName', 'instance of')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('name', 'subclass of'), ('obj', 'http://www.wikidata.org/entity/Q15416'), ('objName', 'television program')]
[('p', 'http://www.wikidata.org/prop/direct/P1889'), ('name', 'different from'), ('obj', 'http://www.wikidata.org/entity/Q1261214'), ('objName', 'television special')]
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('name', 'properties for this type'), ('obj', 'http://www.wikidata.org/entity/P1476'), ('objName', 'title')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('name', 'subclass of'), ('obj', 'http://www.wikidata.org/entity/Q1983062'), ('objName', 'episode')]
[('p', 'http://www.wikidata.org/prop/direct/P1889'), ('name', 'different from'), ('obj', 'http://www.wikidata.org/entity/Q1983062'), ('objName', 'episode')]
[('p', 'http://www.wikid

In [25]:
# P161 cast member
# Q16746256  'Bass Player Wanted'
queryString = """
SELECT DISTINCT ?p ?name ?obj ?objName 
WHERE {
   
   wd:Q16746256 wdt:P161 ?obj .
  
   # get the label
   
   optional{?obj sc:name ?objName.}
   
   
}
ORDER BY ASC (?objName)
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
Empty


In [84]:
#Q13567027 'How I Met Your Mother, season 9'
# P527 has part
 
## Q147235	How I met your mother

queryString = """ 
SELECT DISTINCT  ?seasonName ?epName ?castName
WHERE {
   # bind something
   wd:Q147235 wdt:P527 ?season.
   
   ?season wdt:P527 ?ep. # trovo gli episodi della serie
   
   ?ep wdt:P161 ?cast.
       
  
   
   # get the label
   #?people sc:name ?peopleName.
   ?ep sc:name ?epName.
   ?cast sc:name ?castName.
   ?season sc:name ?seasonName.
   
   
}
ORDER BY ASC (?seasonName )
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('seasonName', 'How I Met Your Mother, season 1'), ('epName', 'Drumroll, Please'), ('castName', 'Virginia Williams')]
[('seasonName', 'How I Met Your Mother, season 1'), ('epName', 'The Wedding'), ('castName', 'Virginia Williams')]
[('seasonName', 'How I Met Your Mother, season 1'), ('epName', 'Nothing Good Happens After 2 A.M.'), ('castName', 'Cobie Smulders')]
[('seasonName', 'How I Met Your Mother, season 1'), ('epName', 'Okay Awesome'), ('castName', 'Cobie Smulders')]
[('seasonName', 'How I Met Your Mother, season 1'), ('epName', 'Return of the Shirt'), ('castName', 'Cobie Smulders')]
[('seasonName', 'How I Met Your Mother, season 1'), ('epName', 'Pilot'), ('castName', 'Cobie Smulders')]
[('seasonName', 'How I Met Your Mother, season 1'), ('epName', 'Purple Giraffe'), ('castName', 'Cobie Smulders')]
[('seasonName', 'How I Met Your Mother, season 1'), ('epName', 'Sweet Taste of Liberty'), ('castName', 'Cobie Smulders')]
[('seasonName', 'How I Met Your Mother, season 1'), ('

Final query for this task

In [95]:
# write your final query
# P527 has part
# Q2472427 
## Q147235	How I met your mother

queryString = """ 
SELECT DISTINCT ?cast ?castName (count(distinct ?ep) as ?countEp)
WHERE {
   # bind something
   wd:Q147235 wdt:P527 ?season.
   
   ?season wdt:P527 ?ep. # trovo gli episodi della serie
   
   ?ep wdt:P161 ?cast.
       
  
   
   # get the label
   #?people sc:name ?peopleName.
   ?ep sc:name ?epName.
   ?cast sc:name ?castName.
   ?season sc:name ?seasonName.
   
   
}

group by ?cast ?castName
order by ?castName
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('cast', 'http://www.wikidata.org/entity/Q8349'), ('castName', '"Weird Al" Yankovic'), ('countEp', '1')]
[('cast', 'http://www.wikidata.org/entity/Q667979'), ('castName', 'Aaron Hill'), ('countEp', '1')]
[('cast', 'http://www.wikidata.org/entity/Q321131'), ('castName', 'Abigail Spencer'), ('countEp', '1')]
[('cast', 'http://www.wikidata.org/entity/Q151866'), ('castName', 'Adriana Lima'), ('countEp', '1')]
[('cast', 'http://www.wikidata.org/entity/Q2827976'), ('castName', 'Ailsa Marshall'), ('countEp', '1')]
[('cast', 'http://www.wikidata.org/entity/Q2636696'), ('castName', 'Alan Fudge'), ('countEp', '1')]
[('cast', 'http://www.wikidata.org/entity/Q277895'), ('castName', 'Alan Thicke'), ('countEp', '3')]
[('cast', 'http://www.wikidata.org/entity/Q16729414'), ('castName', 'Alec Gray'), ('countEp', '1')]
[('cast', 'http://www.wikidata.org/entity/Q366322'), ('castName', 'Alexis Denisof'), ('countEp', '5')]
[('cast', 'http://www.wikidata.org/entity/Q199927'), ('castName', 'Alyson H

## Task 4
Check who is the actor who acted in more films while working on "How I met your mother" (the result set must be a list of triples actor/actress IRI, label and #films).

In [81]:
# write your queries
# troviamo la lista dei film 
# P161  'cast member' 
# wd:Q147235 HIMYM
queryString = """
SELECT DISTINCT  ?actor ?actorName
WHERE {
   
   
   wd:Q147235 wdt:P161 ?actor.
  
  
}
LIMIT 10
"""

print("Results")
x=run_query(queryString)




Results
[('actor', 'http://www.wikidata.org/entity/Q465556')]
[('actor', 'http://www.wikidata.org/entity/Q269891')]
[('actor', 'http://www.wikidata.org/entity/Q4888924')]
[('actor', 'http://www.wikidata.org/entity/Q446031')]
[('actor', 'http://www.wikidata.org/entity/Q312705')]
[('actor', 'http://www.wikidata.org/entity/Q469579')]
[('actor', 'http://www.wikidata.org/entity/Q516659')]
[('actor', 'http://www.wikidata.org/entity/Q522856')]
[('actor', 'http://www.wikidata.org/entity/Q200566')]
[('actor', 'http://www.wikidata.org/entity/Q435839')]
10


In [85]:
# write your queries
# troviamo la lista dei film 
# P161  'cast member' 
# wd:Q147235 HIMYM


queryString = """
SELECT DISTINCT  ?actor ?actorName  ?film ?filmName
WHERE {
   
   
   wd:Q147235 wdt:P161 ?actor.
   ?film wdt:P161 ?actor.
  
  ?actor sc:name ?actorName.
  ?film sc:name ?filmName.
  
}
group by ?actor ?actorName
LIMIT 10
"""

print("Results")
x=run_query(queryString)


Results
[('actor', 'http://www.wikidata.org/entity/Q50737052'), ('actorName', 'K.T. Tatara'), ('film', 'http://www.wikidata.org/entity/Q147235'), ('filmName', 'How I Met Your Mother')]
[('actor', 'http://www.wikidata.org/entity/Q167696'), ('actorName', 'Nicole Scherzinger'), ('film', 'http://www.wikidata.org/entity/Q327713'), ('filmName', 'Men in Black 3')]
[('actor', 'http://www.wikidata.org/entity/Q2633899'), ('actorName', 'Erin Cahill'), ('film', 'http://www.wikidata.org/entity/Q6176596'), ('filmName', 'Jekyll')]
[('actor', 'http://www.wikidata.org/entity/Q485310'), ('actorName', 'Neil Patrick Harris'), ('film', 'http://www.wikidata.org/entity/Q3480575'), ('filmName', 'Return of the Shirt')]
[('actor', 'http://www.wikidata.org/entity/Q44442'), ('actorName', 'Dan Castellaneta'), ('film', 'http://www.wikidata.org/entity/Q797839'), ('filmName', 'Baby Daddy')]
[('actor', 'http://www.wikidata.org/entity/Q433355'), ('actorName', 'Patricia Belcher'), ('film', 'http://www.wikidata.org/entit

In [99]:
# query example

queryString = """
SELECT DISTINCT ?p ?name ?obj
WHERE {
   # bind something
   wd:Q147235 ?p ?obj .
   # get the label
   ?p sc:name ?name.
   filter regex (?name , "start time").
   
   
}
LIMIT 1000
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P580'), ('name', 'start time'), ('obj', '2005-09-19T00:00:00Z')]
1


In [100]:
# query example

queryString = """
SELECT DISTINCT ?p ?name ?obj
WHERE {
   # bind something
   wd:Q147235 ?p ?obj .
   # get the label
   ?p sc:name ?name.
   filter regex (?name , "end time").
   
   
}
LIMIT 1000
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P582'), ('name', 'end time'), ('obj', '2014-03-31T00:00:00Z')]
1


Final query for this task

In [101]:
# P582 'end time'
# P580 'start time'
# write your queries
# troviamo la lista dei film 
# P161  'cast member' 
# wd:Q147235 HIMYM


queryString = """
SELECT DISTINCT  ?actor ?actorName  ?film ?filmName
WHERE {
   
   
   wd:Q147235 wdt:P161 ?actor.
   ?film wdt:P161 ?actor;
       wdt:P582 ?endTimeFilm.
   wd:Q147235 wdt:P580 ?startTime;
       wdt:P582 ?endTime.
    
  
  ?actor sc:name ?actorName.
  ?film sc:name ?filmName.
  
  filter (?startTime < ?endTimeFilm && ?endTime > ?endTimeFilm).
  
}
group by ?actor ?actorName
LIMIT 10
"""

print("Results")
x=run_query(queryString)


Results
[('actor', 'http://www.wikidata.org/entity/Q3306283'), ('actorName', 'Dwight Hicks'), ('film', 'http://www.wikidata.org/entity/Q733960'), ('filmName', 'Cold Case')]
[('actor', 'http://www.wikidata.org/entity/Q289330'), ('actorName', 'Erin Cardillo'), ('film', 'http://www.wikidata.org/entity/Q139849'), ('filmName', 'The Suite Life on Deck')]
[('actor', 'http://www.wikidata.org/entity/Q231091'), ('actorName', 'Morena Baccarin'), ('film', 'http://www.wikidata.org/entity/Q470111'), ('filmName', 'Medium')]
[('actor', 'http://www.wikidata.org/entity/Q469579'), ('actorName', 'Mircea Monroe'), ('film', 'http://www.wikidata.org/entity/Q2293'), ('filmName', 'Chuck')]
[('actor', 'http://www.wikidata.org/entity/Q3943077'), ('actorName', 'Ryan Michelle Bathe'), ('film', 'http://www.wikidata.org/entity/Q283393'), ('filmName', 'Private Practice')]
[('actor', 'http://www.wikidata.org/entity/Q2627090'), ('actorName', 'Dave Thomas'), ('film', 'http://www.wikidata.org/entity/Q23609'), ('filmName'

In [102]:
# P582 'end time'
# P580 'start time'
# write your queries
# troviamo la lista dei film 
# P161  'cast member' 
# wd:Q147235 HIMYM


queryString = """
SELECT DISTINCT  ?actor ?actorName  (count (?film) as ?numFilm )
WHERE {
   
   
   wd:Q147235 wdt:P161 ?actor.
   ?film wdt:P161 ?actor;
       wdt:P582 ?endTimeFilm.
   wd:Q147235 wdt:P580 ?startTime;
       wdt:P582 ?endTime.
    
  
  ?actor sc:name ?actorName.
  ?film sc:name ?filmName.
  
  filter (?startTime < ?endTimeFilm && ?endTime > ?endTimeFilm).
  
}
group by ?actor ?actorName
LIMIT 10
"""

print("Results")
x=run_query(queryString)



Results
[('actor', 'http://www.wikidata.org/entity/Q3306283'), ('actorName', 'Dwight Hicks'), ('numFilm', '1')]
[('actor', 'http://www.wikidata.org/entity/Q234152'), ('actorName', 'Becki Newton'), ('numFilm', '3')]
[('actor', 'http://www.wikidata.org/entity/Q233042'), ('actorName', 'Nancy Travis'), ('numFilm', '2')]
[('actor', 'http://www.wikidata.org/entity/Q635974'), ('actorName', 'Michael Trucco'), ('numFilm', '5')]
[('actor', 'http://www.wikidata.org/entity/Q926420'), ('actorName', 'Robbie Amell'), ('numFilm', '1')]
[('actor', 'http://www.wikidata.org/entity/Q1459918'), ('actorName', 'James Tupper'), ('numFilm', '4')]
[('actor', 'http://www.wikidata.org/entity/Q1806933'), ('actorName', 'Rhys Darby'), ('numFilm', '1')]
[('actor', 'http://www.wikidata.org/entity/Q73346'), ('actorName', 'Ralph Macchio'), ('numFilm', '1')]
[('actor', 'http://www.wikidata.org/entity/Q2842824'), ('actorName', 'America Olivo'), ('numFilm', '1')]
[('actor', 'http://www.wikidata.org/entity/Q463673'), ('acto

## Task 5
Compare HIMYM with the tv series "The Office (US)" in terms of number of seasons, episods and cast members (the result set must be two elements -one for each tv series- of tv series IRI, label, #seasons, #episodes and #cast members).

In [2]:
# write your queries
#wd:Q23831` | The Office (US)  

# Q147235	How I met your mother


queryString = """
SELECT * WHERE
{ 
{
    SELECT DISTINCT (count (?obj) as ?numSeasonHIMYM)
            WHERE {
               # bind something
               wd:Q23831 ?p ?obj .


               # get the label
               ?p sc:name ?name
               optional{?obj sc:name ?objName.}
               filter regex(?objName, "season").

            }
            
            LIMIT 20
}
UNION
{
    SELECT DISTINCT (count (?obj) as ?numSeasonTheOffice)
            WHERE {
               # bind something
               wd:Q147235 ?p ?obj .


               # get the label
               ?p sc:name ?name
               optional{?obj sc:name ?objName.}
               filter regex(?objName, "season").

            }
            
            LIMIT 20
}
}



"""

print("Results")
x=run_query(queryString)


Results
[('numSeasonHIMYM', '9')]
[('numSeasonTheOffice', '9')]
2


In [3]:
# write your queries
#wd:Q23831` | The Office (US)  

# Q147235	How I met your mother


queryString = """
SELECT * WHERE
{ 
{
    SELECT ?IRI ?HIMYM (count (?obj) as ?numSeasonHIMYM)
            WHERE {
               # bind something
               wd:Q147235 ?p ?obj .


               # get the label
               
               ?p sc:name ?name.
               wd:Q147235 sc:name ?HIMYM.
               
               optional{?obj sc:name ?objName.}
               filter regex(?objName, "season").
               
               BIND ((wd:Q147235) AS ?IRI).

            }
            GROUP BY ?HIMYM ?IRI
            LIMIT 20
            
}
UNION
{
    SELECT ?IRI ?TheOffice (count (?obj) as ?numSeasonTheOffice)
            WHERE {
               # bind something
               wd:Q23831 ?p ?obj .


               # get the label
               ?p sc:name ?name.
               wd:Q23831 sc:name ?TheOffice.
               
               optional{?obj sc:name ?objName.}
               filter regex(?objName, "season").
               
               BIND ((wd:Q23831) AS ?IRI).

            }
            GROUP BY ?TheOffice ?IRI
            LIMIT 20
            
}
}
"""

print("Results")
x=run_query(queryString)



Results
[('IRI', 'http://www.wikidata.org/entity/Q147235'), ('HIMYM', 'How I Met Your Mother'), ('numSeasonHIMYM', '9')]
[('IRI', 'http://www.wikidata.org/entity/Q23831'), ('TheOffice', 'The Office'), ('numSeasonTheOffice', '9')]
2


In [14]:
# write your queries
#wd:Q23831` | The Office (US)  

# Q147235	How I met your mother

#P527 'has part'


queryString = """
SELECT * WHERE
{ 
{
    SELECT ?IRI ?HIMYM (count (?season) as ?numSeasonHIMYM) (count (?ep) as ?epCount)
            WHERE {
               
               wd:Q147235 wdt:P527 ?season.
               ?season wdt:P527 ?ep.

              
               wd:Q147235 sc:name ?HIMYM.
               BIND ((wd:Q147235) AS ?IRI).

            }
            GROUP BY ?HIMYM ?IRI
            LIMIT 20
            
}
UNION
{
    SELECT ?IRI ?TheOffice (count (?obj) as ?numSeasonTheOffice)
            WHERE {
               # bind something
               wd:Q23831 ?p ?obj .


               # get the label
               ?p sc:name ?name.
               wd:Q23831 sc:name ?TheOffice.
               
               optional{?obj sc:name ?objName.}
               filter regex(?objName, "season").
               
               BIND ((wd:Q23831) AS ?IRI).

            }
            GROUP BY ?TheOffice ?IRI
            LIMIT 20
            
}
}
"""

print("Results")
x=run_query(queryString)




Results
[('IRI', 'http://www.wikidata.org/entity/Q147235'), ('HIMYM', 'How I Met Your Mother'), ('numSeasonHIMYM', '208'), ('epCount', '208')]
[('IRI', 'http://www.wikidata.org/entity/Q23831'), ('TheOffice', 'The Office'), ('numSeasonTheOffice', '9')]
2


Final query for this task

In [15]:
# write your final query
# write your queries
#wd:Q23831` | The Office (US)  

# Q147235	How I met your mother

#P527 'has part'


queryString = """
SELECT * WHERE
{ 
{
    SELECT ?IRI ?HIMYM (count (?season) as ?numSeasonHIMYM) (count (?ep) as ?epCount)
            WHERE {
               
               wd:Q147235 wdt:P527 ?season.
               ?season wdt:P527 ?ep.

              
               wd:Q147235 sc:name ?HIMYM.
               BIND ((wd:Q147235) AS ?IRI).

            }
            GROUP BY ?HIMYM ?IRI
            LIMIT 20
            
}
UNION
{
    SELECT ?IRI ?TheOffice (count (?season) as ?numSeasonTheOffice)(count (?ep) as ?epCount)
            WHERE {
              
               wd:Q23831 ?p ?season.
               ?season wdt:P527 ?ep.

               wd:Q23831 sc:name ?TheOffice.
               BIND ((wd:Q23831) AS ?IRI).

            }
            GROUP BY ?TheOffice ?IRI
            LIMIT 20
            
}
}
"""

print("Results")
x=run_query(queryString)

Results
[('IRI', 'http://www.wikidata.org/entity/Q147235'), ('HIMYM', 'How I Met Your Mother'), ('numSeasonHIMYM', '208'), ('epCount', '208')]
[('IRI', 'http://www.wikidata.org/entity/Q23831'), ('epCount', '216'), ('TheOffice', 'The Office'), ('numSeasonTheOffice', '216')]
2


## Task 6
Return how many of the actors who are members of the cast of the tv series have [Kavin Bacon number](https://en.wikipedia.org/wiki/Six_Degrees_of_Kevin_Bacon#:~:text=Kevin%20Bacon%20himself%20has%20a,Bacon%20number%20is%20N%2B1.) equal to 2 (the result set must be a list of couples actor/actress IRI and label).

In [20]:
# Return a list of actor of the cast

# P161 cast member
# Q147235  'HIMYM'
queryString = """
SELECT DISTINCT ?actor ?objName 
WHERE {
   
   wd:Q147235 wdt:P161 ?actor .
  

   ?actor sc:name ?objName.
   
}
ORDER BY ASC (?objName)
LIMIT 10
"""

print("Results")
x=run_query(queryString)

Results
[('actor', 'http://www.wikidata.org/entity/Q8349'), ('objName', '"Weird Al" Yankovic')]
[('actor', 'http://www.wikidata.org/entity/Q24576840'), ('objName', 'Aaron Hendry')]
[('actor', 'http://www.wikidata.org/entity/Q667979'), ('objName', 'Aaron Hill')]
[('actor', 'http://www.wikidata.org/entity/Q2708237'), ('objName', 'Abby Elliott')]
[('actor', 'http://www.wikidata.org/entity/Q50736659'), ('objName', 'Abhi Sinha')]
[('actor', 'http://www.wikidata.org/entity/Q321131'), ('objName', 'Abigail Spencer')]
[('actor', 'http://www.wikidata.org/entity/Q50736705'), ('objName', 'Adam Paul')]
[('actor', 'http://www.wikidata.org/entity/Q151866'), ('objName', 'Adriana Lima')]
[('actor', 'http://www.wikidata.org/entity/Q2827706'), ('objName', "Ahna O'Reilly")]
[('actor', 'http://www.wikidata.org/entity/Q2827976'), ('objName', 'Ailsa Marshall')]
10


In [29]:
# trovare Kevin Bacon
# Return a list of actor of the cast

# P161 cast member
# Q147235  'HIMYM'
queryString = """
SELECT DISTINCT ?actor ?nameActor 
WHERE {
   
    ?film wdt:P161 ?actor .
  

   ?actor sc:name ?nameActor.
   filter regex(?nameActor,"Kevin Bacon").
   
}
ORDER BY ASC (?objName)
LIMIT 10
"""

print("Results")
x=run_query(queryString)


Results
[('actor', 'http://www.wikidata.org/entity/Q3454165'), ('nameActor', 'Kevin Bacon')]
1


In [32]:
#Q3454165 Kevin Bacon
# film di kevin bacon
#lista attori con keni bacon 1
queryString = """
SELECT DISTINCT ?film ?filmName ?actor ?nameActor
WHERE {
   
    ?film wdt:P161 wd:Q3454165;
        wdt:P161 ?actor.
    
  

   ?film sc:name ?filmName.
   ?actor sc:name ?nameActor.
   
}
ORDER BY ASC (?objName)
LIMIT 10
"""

print("Results")
x=run_query(queryString)

Results
[('film', 'http://www.wikidata.org/entity/Q48742976'), ('filmName', 'City on a Hill'), ('actor', 'http://www.wikidata.org/entity/Q15867224'), ('nameActor', "Mark O'Brien")]
[('film', 'http://www.wikidata.org/entity/Q1189631'), ('filmName', 'The Following'), ('actor', 'http://www.wikidata.org/entity/Q16185788'), ('nameActor', 'Kyle Catlett')]
[('film', 'http://www.wikidata.org/entity/Q519490'), ('filmName', 'Crazy, Stupid, Love.'), ('actor', 'http://www.wikidata.org/entity/Q269891'), ('nameActor', 'Julianna Guill')]
[('film', 'http://www.wikidata.org/entity/Q16971610'), ('filmName', 'Black Mass'), ('actor', 'http://www.wikidata.org/entity/Q391359'), ('nameActor', 'David Harbour')]
[('film', 'http://www.wikidata.org/entity/Q611535'), ('filmName', 'Novocaine'), ('actor', 'http://www.wikidata.org/entity/Q449580'), ('nameActor', 'Lynne Thigpen')]
[('film', 'http://www.wikidata.org/entity/Q990987'), ('filmName', 'My One and Only'), ('actor', 'http://www.wikidata.org/entity/Q918866'),

In [40]:
#Q3454165 Kevin Bacon
# film di kevin bacon
#lista attori con keni bacon 2
# Q147235	How I met your mother


queryString = """
SELECT DISTINCT ?actor2 ?nameActor2
WHERE {
   
    ?film wdt:P161 wd:Q3454165;
        wdt:P161 ?actor1.
    
  
     wd:Q147235 wdt:P161 ?actor2.
     ?film wdt:P161 ?actor1;
         wdt:P161 ?actor2.
     
    
   ?actor2 sc:name ?nameActor2.
   
   ?film sc:name ?filmName.
   ?actor1 sc:name ?nameActor.
   
}
LIMIT 10
"""

print("Results")
x=run_query(queryString)

Results
[('film', 'http://www.wikidata.org/entity/Q519490'), ('filmName', 'Crazy, Stupid, Love.'), ('actor2', 'http://www.wikidata.org/entity/Q269891'), ('nameActor2', 'Julianna Guill')]
[('film', 'http://www.wikidata.org/entity/Q370893'), ('filmName', 'The Air I Breathe'), ('actor2', 'http://www.wikidata.org/entity/Q312705'), ('nameActor2', 'John Cho')]
[('film', 'http://www.wikidata.org/entity/Q16167570'), ('filmName', 'Skum Rocks!'), ('actor2', 'http://www.wikidata.org/entity/Q234715'), ('nameActor2', 'Jamie-Lynn Sigler')]
[('film', 'http://www.wikidata.org/entity/Q627533'), ('filmName', 'Footloose'), ('actor2', 'http://www.wikidata.org/entity/Q311271'), ('nameActor2', 'John Lithgow')]
[('film', 'http://www.wikidata.org/entity/Q1476932'), ('filmName', 'Queens Logic'), ('actor2', 'http://www.wikidata.org/entity/Q234137'), ('nameActor2', 'Megan Mullally')]
[('film', 'http://www.wikidata.org/entity/Q223596'), ('filmName', 'X-Men: First Class'), ('actor2', 'http://www.wikidata.org/entit

Final query for this task

In [56]:
# write your final query
#vado a togliere gli attori che hanno il numero 1 di kevin Bacon

#Q3454165 Kevin Bacon
# film di kevin bacon
#lista attori con keni bacon 2
# Q147235	How I met your mother


queryString = """
SELECT DISTINCT ?actor2 ?nameActor2
WHERE {
   
   
    ?film1 wdt:P161 wd:Q3454165;
        wdt:P161 ?actor1.
    
  
     wd:Q147235 wdt:P161 ?actor2.
     ?film wdt:P161 ?actor1;
         wdt:P161 ?actor2.
     
    
   ?actor2 sc:name ?nameActor2.
   
   ?film sc:name ?filmName.
   ?actor1 sc:name ?nameActor.
   
   FILTER(?actor1 != ?actor2).
  
 
   
}
LIMIT 10
"""

print("Results")
x=run_query(queryString)

Results
[('actor2', 'http://www.wikidata.org/entity/Q566037'), ('nameActor2', 'Scoot McNairy')]
[('actor2', 'http://www.wikidata.org/entity/Q1992967'), ('nameActor2', 'John Getz')]
[('actor2', 'http://www.wikidata.org/entity/Q260184'), ('nameActor2', 'Eva Amurri')]
[('actor2', 'http://www.wikidata.org/entity/Q450496'), ('nameActor2', 'Alyssa Shafer')]
[('actor2', 'http://www.wikidata.org/entity/Q231006'), ('nameActor2', 'Jayma Mays')]
[('actor2', 'http://www.wikidata.org/entity/Q454698'), ('nameActor2', 'Darcy Rose Byrnes')]
[('actor2', 'http://www.wikidata.org/entity/Q312705'), ('nameActor2', 'John Cho')]
[('actor2', 'http://www.wikidata.org/entity/Q4888924'), ('nameActor2', 'Benjamin Koldyke')]
[('actor2', 'http://www.wikidata.org/entity/Q235740'), ('nameActor2', 'Busy Philipps')]
[('actor2', 'http://www.wikidata.org/entity/Q943613'), ('nameActor2', 'Samm Levine')]
10


## Task 7
Consider the actors who are members of the cast of HIMYM. Amongst the tv series which these actors acted return only those which received more than 2 awards (the result set must be triples of tv series IRI, label, #awards won).

In [58]:
# P161 cast member
# Q147235  'HIMYM'
# Q5398426 'television series'
queryString = """
SELECT DISTINCT ?televisionSeries ?televisionSeriesName
WHERE {
   
   wd:Q147235 wdt:P161 ?actor.
   ?televisionSeries wdt:P161 ?actor;
       ?type wd:Q5398426.
  

   ?televisionSeries sc:name ?televisionSeriesName.
   
}
ORDER BY ASC (?objName)
LIMIT 10
"""

print("Results")
x=run_query(queryString)

Results
[('televisionSeries', 'http://www.wikidata.org/entity/Q500769'), ('televisionSeriesName', 'Freaks and Geeks')]
[('televisionSeries', 'http://www.wikidata.org/entity/Q17049828'), ('televisionSeriesName', 'Bad Samaritans')]
[('televisionSeries', 'http://www.wikidata.org/entity/Q15072819'), ('televisionSeriesName', 'Halt and Catch Fire')]
[('televisionSeries', 'http://www.wikidata.org/entity/Q15931555'), ('televisionSeriesName', 'Fargo')]
[('televisionSeries', 'http://www.wikidata.org/entity/Q18151386'), ('televisionSeriesName', 'Kirby Buckets')]
[('televisionSeries', 'http://www.wikidata.org/entity/Q643894'), ('televisionSeriesName', 'Suddenly Susan')]
[('televisionSeries', 'http://www.wikidata.org/entity/Q111756'), ('televisionSeriesName', "Kickin' It")]
[('televisionSeries', 'http://www.wikidata.org/entity/Q22908690'), ('televisionSeriesName', 'The Good Place')]
[('televisionSeries', 'http://www.wikidata.org/entity/Q22908828'), ('televisionSeriesName', 'Haters Back Off')]
[('te

In [76]:


# P161 cast member
# Q147235  'HIMYM'
# Q5398426 'television series'
queryString = """
SELECT DISTINCT ?p ?pName ?o ?oName
WHERE {
   
   wd:Q147235 wdt:P161 ?actor.
   ?televisionSeries wdt:P161 ?actor;
       ?type wd:Q5398426.
    ?televisionSeries ?p ?o.
  

   ?televisionSeries sc:name ?televisionSeriesName.
   ?p sc:name ?pName.
   ?o sc:name ?oName.
   
}
ORDER BY ASC (?objName)
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P136'), ('pName', 'genre'), ('o', 'http://www.wikidata.org/entity/Q7168625'), ('oName', 'historical drama')]
[('p', 'http://www.wikidata.org/prop/direct/P136'), ('pName', 'genre'), ('o', 'http://www.wikidata.org/entity/Q157443'), ('oName', 'comedy film')]
[('p', 'http://www.wikidata.org/prop/direct/P136'), ('pName', 'genre'), ('o', 'http://www.wikidata.org/entity/Q859369'), ('oName', 'comedy-drama')]
[('p', 'http://www.wikidata.org/prop/direct/P136'), ('pName', 'genre'), ('o', 'http://www.wikidata.org/entity/Q102430221'), ('oName', 'coming-of-age television program')]
[('p', 'http://www.wikidata.org/prop/direct/P136'), ('pName', 'genre'), ('o', 'http://www.wikidata.org/entity/Q775344'), ('oName', 'teen drama')]
[('p', 'http://www.wikidata.org/prop/direct/P1411'), ('pName', 'nominated for'), ('o', 'http://www.wikidata.org/entity/Q3045762'), ('oName', 'Primetime Emmy Award for Outstanding Writing for a Comedy Series')]
[('p', 'http://ww

In [62]:
# P1411 'nominated for'
# P161 cast member
# Q147235  'HIMYM'
# Q5398426 'television series'
queryString = """
SELECT DISTINCT ?televisionSeries ?televisionSeriesName (count (?award) as ?award)
WHERE {
   
   wd:Q147235 wdt:P161 ?actor.
   ?televisionSeries wdt:P161 ?actor;
       ?type wd:Q5398426.
    ?televisionSeries wdt:P1411 ?award.
  

   ?televisionSeries sc:name ?televisionSeriesName.
   
  
   
}
LIMIT 10
"""

print("Results")
x=run_query(queryString)

Results
[('televisionSeries', 'http://www.wikidata.org/entity/Q1064766'), ('televisionSeriesName', 'Mad TV'), ('award', '8')]
[('televisionSeries', 'http://www.wikidata.org/entity/Q1571652'), ('televisionSeriesName', 'Dog with a Blog'), ('award', '1')]
[('televisionSeries', 'http://www.wikidata.org/entity/Q117590'), ('televisionSeriesName', 'Beverly Hills, 90210'), ('award', '30')]
[('televisionSeries', 'http://www.wikidata.org/entity/Q184342'), ('televisionSeriesName', 'Wizards of Waverly Place'), ('award', '4')]
[('televisionSeries', 'http://www.wikidata.org/entity/Q1061246'), ('televisionSeriesName', 'Growing Pains'), ('award', '2')]
[('televisionSeries', 'http://www.wikidata.org/entity/Q1584299'), ('televisionSeriesName', 'Happy Endings'), ('award', '1')]
[('televisionSeries', 'http://www.wikidata.org/entity/Q23609'), ('televisionSeriesName', 'Weeds'), ('award', '240')]
[('televisionSeries', 'http://www.wikidata.org/entity/Q139849'), ('televisionSeriesName', 'The Suite Life on Deck

Final query for this task

In [75]:
# P1411 'nominated for'
# P161 cast member
# Q147235  'HIMYM'
# Q5398426 'television series'
queryString = """
SELECT DISTINCT ?televisionSeries ?televisionSeriesName (count (?award) as ?awards)
WHERE {
   
   wd:Q147235 wdt:P161 ?actor.
   ?televisionSeries wdt:P161 ?actor;
       ?type wd:Q5398426.
    ?televisionSeries wdt:P1411 ?award.
  

   ?televisionSeries sc:name ?televisionSeriesName.
 
  
   
}
GROUP BY ?televisionSeries ?televisionSeriesName
HAVING (count (?award) > 1)
LIMIT 10
"""

print("Results")
x=run_query(queryString)

Results
[('televisionSeries', 'http://www.wikidata.org/entity/Q1066201'), ('televisionSeriesName', 'Make It or Break It'), ('awards', '6')]
[('televisionSeries', 'http://www.wikidata.org/entity/Q564345'), ('televisionSeriesName', 'Person of Interest'), ('awards', '16')]
[('televisionSeries', 'http://www.wikidata.org/entity/Q187239'), ('televisionSeriesName', 'Zeke and Luther'), ('awards', '2')]
[('televisionSeries', 'http://www.wikidata.org/entity/Q366375'), ('televisionSeriesName', 'Good luck, Charlie!'), ('awards', '2')]
[('televisionSeries', 'http://www.wikidata.org/entity/Q193462'), ('televisionSeriesName', 'Gilmore Girls'), ('awards', '156')]
[('televisionSeries', 'http://www.wikidata.org/entity/Q386880'), ('televisionSeriesName', 'Castle'), ('awards', '78')]
[('televisionSeries', 'http://www.wikidata.org/entity/Q1027097'), ('televisionSeriesName', 'Call Me Fitz'), ('awards', '2')]
[('televisionSeries', 'http://www.wikidata.org/entity/Q390405'), ('televisionSeriesName', 'Nikita'),